In [35]:
import org.apache.spark.sql.types._
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions._
val sparkSession = spark
import sparkSession.implicits._

sparkSession = org.apache.spark.sql.SparkSession@14ec6ada


Spark local-1533526251577: Some(http://192.168.1.7:4040)

In [2]:
def fixEncoding(text:String): String = {
    val regex = "[\\xc2-\\xf4][\\x80-\\xbf]+".r
    return regex.replaceAllIn(text, m => new String(m.group(0).getBytes("ISO-8859-1"),"UTF-8"))
}

fixEncoding: (text: String)String


In [15]:
// case class Reddit(author: String, author_flair_css_class: String, author_flair_text: String, body: String, controversiality: String, created_utc: String, distinguished: String, edited: String, gilded: String, id: String, link_id: String, parent_id: String, retrieved_on: String, score: String, stickied: String, subreddit: String, subreddit_id: String, ups: String)
case class Reddit(author: String, body: String, author_flair_text: String, gilded: BigInt, score: BigInt,
                  link_id: String, retrieved_on: Long, author_flair_css_class: String, subreddit: String,
                  edited: String, ups: BigInt, controversiality: BigInt, created_utc: Long,
                  parent_id: String, subreddit_id: String, id: String, distinguished: String)

defined class Reddit


lastException: Throwable = null


In [16]:
val redditDF = spark.read.json("RC_2005-12.json")

redditDF = [author: string, author_flair_css_class: string ... 16 more fields]


[author: string, author_flair_css_class: string ... 16 more fields]

In [17]:
val redditDS = redditDF.as[Reddit]
val postsFiltered = redditDS.filter(a => a.author != "[deleted]" && a.author != "AutoModerator")
val authorsSubreddits = postsFiltered.select($"author", $"body", $"subreddit").groupBy("author","subreddit").count().sort($"count".desc).show()

+--------------+----------+-----+
|        author| subreddit|count|
+--------------+----------+-----+
|      symbiont|reddit.com|   21|
|  michaelneale|reddit.com|   21|
|      mattknox|reddit.com|   19|
|           chu|reddit.com|   16|
|    paulgraham|reddit.com|   15|
|           e40|reddit.com|   13|
|      fnord123|reddit.com|   12|
|         jcage|reddit.com|   10|
|        TronXD|reddit.com|    9|
|       beza1e1|reddit.com|    9|
|        stesch|reddit.com|    9|
|        rdtiii|reddit.com|    9|
|      enjahova|reddit.com|    9|
|       bugbear|reddit.com|    9|
|      JimThome|reddit.com|    8|
|JonathanGCohen|reddit.com|    8|
|     lynxcat73|reddit.com|    8|
|          spez|reddit.com|    7|
|     Schwallex|reddit.com|    7|
|     drewyates|reddit.com|    7|
+--------------+----------+-----+
only showing top 20 rows



redditDS = [author: string, author_flair_css_class: string ... 16 more fields]
postsFiltered = [author: string, author_flair_css_class: string ... 16 more fields]


authorsSubreddits: Unit = ()


[author: string, author_flair_css_class: string ... 16 more fields]

In [38]:
val subredditsCounter = postsFiltered.select($"author", $"subreddit").
                        groupBy("subreddit").agg(count("subreddit") as "total posts").sort($"total posts".desc).show()

+----------+-----------+
| subreddit|total posts|
+----------+-----------+
|reddit.com|        890|
+----------+-----------+



lastException: Throwable = null
subredditsCounter: Unit = ()


In [47]:
val upvotesCounter = postsFiltered.select($"author", $"subreddit", $"ups").
                     groupBy("author").agg(sum("ups") as "total ups").sort($"total ups".desc).show()

+-------------+---------+
|       author|total ups|
+-------------+---------+
|   paulgraham|       98|
|     mattknox|       77|
|      bugbear|       43|
| michaelneale|       39|
|     dstowell|       38|
|         spez|       36|
|      AaronSw|       30|
|        sempf|       29|
|     JimThome|       29|
|    bolinfest|       28|
|       davidw|       27|
|          Zak|       26|
|     symbiont|       25|
|     binladen|       24|
|         cg84|       23|
|     enjahova|       23|
|brendankohler|       21|
|       dylanm|       20|
|     beastboy|       19|
|    Schwallex|       18|
+-------------+---------+
only showing top 20 rows



upvotesCounter: Unit = ()


In [46]:
val downvotesCounter = postsFiltered.filter(a => a.ups < 0).select($"author", $"subreddit", $"ups").
                       groupBy("author").agg(sum("ups") as "total downs").sort($"total downs".asc).show()

+---------------+-----------+
|         author|total downs|
+---------------+-----------+
|      ssundar78|        -21|
|          zimba|        -18|
|       dguertin|        -14|
|         TronXD|        -14|
|        Taladar|        -11|
|         guytam|         -8|
|        jedberg|         -8|
|          alecb|         -7|
|     hartshorne|         -6|
|        flabbah|         -6|
|       fnord123|         -6|
|     goodgoblin|         -5|
|       mckirkus|         -5|
|      lynxcat73|         -5|
| Godfrey_Daniel|         -4|
|theycallmemorty|         -4|
|            chu|         -4|
| BJThunderstone|         -4|
|          Donut|         -4|
|      priestess|         -3|
+---------------+-----------+
only showing top 20 rows



downvotesCounter: Unit = ()
